In [2]:
import scipy, pylab, math, numpy as np, sympy, matplotlib, matplotlib.pyplot as plt, matplotlib.pylab as plt2, networkx as nx;
import seaborn as sns ### plotly.plotly as py, plotly.graph_objs as go, plotly.tools as tls
from __future__ import division
from sympy.printing import latex
from IPython.display import set_matplotlib_formats
from ipywidgets import interact
import bokeh.io as bio
from bokeh.plotting import figure, show, output_notebook #, output_file
from bokeh.models import HoverTool, Line, TapTool, CustomJS, ColumnDataSource

set_matplotlib_formats('png', 'pdf')
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
matplotlib.rcParams.update({'font.size': 12, 'font.family': 'calibri', 'mathtext.fontset': 'stix'})
# py.sign_in("joknowle", "h1w8dn6q7h")
# <nbformat>2</nbformat>
# <markdowncell>
#init_printing(use_latex='mathjax')
#init_printing()

In [3]:
x = np.linspace(0, 2*np.pi, 2000)
y = np.sin(x)

source = ColumnDataSource(data=dict(x=x, y=y))
p = figure(title="simple line example", plot_height=300, plot_width=600, y_range=(-5, 5))
p.line(x, y, color="#2222aa", alpha=0.5, line_width=2, source=source, name="foo")

def update(f, w=1, A=1, phi=0):
    if   f == "sin": func = np.sin
    elif f == "cos": func = np.cos
    elif f == "tan": func = np.tan
    source.data['y'] = A * func(w * x + phi)
    bio.push_notebook()

show(p)

In [4]:
from ipywidgets import interact
interact(update, f=["sin", "cos", "tan"], w=(0,10, 0.1), A=(0,5, 0.1), phi=(0, 10, 0.1))

C:\Users\Mahdie\Anaconda3\lib\site-packages\bokeh\io.py:532: UserWarning: Cannot find a last shown plot to update. Call output_notebook() and show() before push_notebook()
  warnings.warn("Cannot find a last shown plot to update. Call output_notebook() and show() before push_notebook()")


<function __main__.update>

In [ ]:
class Graph(nx.DiGraph):
    def draw(self, ax = None, **kwds):
        pos = self.get_pos()
        node_size = kwds.pop('node_size', 200)
        alpha = kwds.pop('alpha', 0.3)
        if ax is None: nx.draw(self, pos = pos,
                               node_size = node_size,
                               alpha = alpha,
                               **kwds)
        else:
            nx.draw(self,pos=pos,
                    node_size=node_size,
                    alpha=alpha,
                    ax = ax,**kwds)
    def get_pos(self,n=None):
        pos = dict([(i, j['pos'] ) for i, j in self.nodes(data = True)])
        if n is None:
            return pos
        else:
            return pos[n]
    def getx(self,x):
        return sorted([(i[0],i[1]['val'] ) for i in self.nodes(True) if i[0][0]==x])

    def gety(self,y):
        return sorted([(i[0],i[1]['val'] ) for i in self.nodes(True) if i[0][1]==y])

# functions to allow diagonal lattice walking
def diagwalk(level, n):
    x = level
    y = -level
    while y <= 1 and x < n+1:
        yield (x, y)
        x += 1
        y += 1
def diagwalker(n):
    'daisy-chains the individual diagonal walkers'
    assert n%2 # odd only
    return it.chain(*(diagwalk(i, n) for i in range(n+1)))

In [ ]:
def construct_graph(level = 51):
    g = Graph()
    g.level = level
    g.add_nodes_from([(i, dict(pos = i)) for i in diagwalker(g.level)])
    
    for x,y in g.nodes():
        if y != 1 and x < g.level:
            g.add_edge((x, y), (x + 1, y - 1) )
            g.add_edge((x, y), (x + 1, y + 1) )
    
    g.node[(0,0)]['val'] = 1 # long int
    for j in diagwalker(g.level):
        if j == (0, 0): continue
        x, y = j
        g.node[j]['val'] = sum(g.node[k]['val'] for k in g.predecessors(j))
    return g
g = construct_graph()

In [ ]:
fig,ax=plt.subplots()
fig.set_size_inches((20,8))
ax.set_aspect(1)
ax.set_title('Directed Path Lattice',fontsize=18)
g.draw(ax=ax)